In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [2]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome()  # Optional argument, if not specified will search path.

driver.get('https://www.redbus.in/');

#time.sleep(5) # Let the user actually see something!
driver.maximize_window()
driver.execute_script("window.scrollBy(0, window.innerHeight);")

#view_all = driver.find_element(By.CSS_SELECTOR, 'a[href="https://www.redbus.in/online-booking/rtc-directory"]')
#view_all.click();

#driver.quit()


  

# 1. KERALA

In [3]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome()  # Optional argument, if not specified will search path.

driver.get('https://www.redbus.in/');

#time.sleep(5) # Let the user actually see something!
driver.maximize_window()
driver.execute_script("window.scrollBy(0, window.innerHeight);")

#view_all = driver.find_element(By.CSS_SELECTOR, 'a[href="https://www.redbus.in/online-booking/rtc-directory"]')
#view_all.click();

#driver.quit()


import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import mysql.connector

driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/rtc-directory")
driver.maximize_window()
kerala = driver.find_element(By.CSS_SELECTOR, 'a[href="/online-booking/ksrtc-kerala"]')
kerala.click();
time.sleep(3)
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")

kerala_route_names = []
kerala_route_links = []

def scrape_data():
    route_names = driver.find_elements(By.CLASS_NAME, "route")
    for route_name in route_names:
        kerala_route_names.append(route_name.text)

    route_links = driver.find_elements(By.CLASS_NAME, "route")
    hrefs = [route_link.get_attribute('href') for route_link in route_links]
    for href in hrefs:
        kerala_route_links.append(href)

page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")
for i in range(len(page_tabs)):
    if i > 0:
        page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")[i]
        driver.execute_script("arguments[0].click();", page_tabs)        
        time.sleep(4)
    scrape_data()



df = pd.DataFrame(columns=['route_name', 'route_link', 'busname', 'bustype', 'departing_time', 
                           'duration', 'reaching_time', 'star_rating', 'price', 'seats_available'])
count = 0
for link in kerala_route_links:
    driver.get(link)
    driver.maximize_window()

    try:
        wait = WebDriverWait(driver, 3)
        wait_button = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "button")))
        driver.execute_script("arguments[0].click();", wait_button)
        # government_buses = driver.find_element(By.CLASS_NAME, "button")
        #driver.execute_script("arguments[0].click();", government_buses)
        
    except NoSuchElementException:
        pass
    except TimeoutException:
        pass    
    
    for t in range(22):     #scrolling 20 times so that whole website loads
        driver.execute_script("window.scrollBy(0, 650);")
        time.sleep(0.3)

    
    route_name = []
    route_link = []
    bus_name = []
    bus_type = []
    departing = []
    duration = []
    reaching = []
    rating = []
    price = []
    availability = []
    dict = {}
    df1 = pd.DataFrame()

    names = driver.find_elements(By.CLASS_NAME,"travels")
    types = driver.find_elements(By.CLASS_NAME,"bus-type")
    depart_timings = driver.find_elements(By.CLASS_NAME,"dp-time")
    travel_durations = driver.find_elements(By.CLASS_NAME,"dur")
    arrivals = driver.find_elements(By.CLASS_NAME,"bp-time")
    star_ratings = driver.find_elements(By.CLASS_NAME,"column-six")
    fares = driver.find_elements(By.CLASS_NAME,"fare")
    seats_available = driver.find_elements(By.CLASS_NAME,"seat-left")

    for name in names:
        bus_name.append(name.text)

    for type in types:
        bus_type.append(type.text)

    for depart_timing in depart_timings:
        departing.append(depart_timing.text)

    for travel_duration in travel_durations:
        duration.append(travel_duration.text)

    for arrival in arrivals:
        reaching.append(arrival.text)

    for star_rating in star_ratings:
        rating.append(star_rating.text[:3])

    for fare in fares:
        if (fare.text).isdigit():
            price.append(fare.text)
        else:
            price.append((fare.text).replace('INR','').strip())

    for seat_available in seats_available:
        availability.append(seat_available.text[:2].strip())
    
    for i in range(len(bus_name)):
        route_name.append(kerala_route_names[count])
        route_link.append(link)

    count += 1




    for i in range(len(rating)):
        if rating[i] == 'New' or rating[i] == ' ':
            rating[i] = '0.0'


    dict = {'route_name':route_name, 'route_link':route_link, 'busname':bus_name, 'bustype':bus_type, 'departing_time':departing,
            'duration':duration, 'reaching_time':reaching, 'star_rating':rating, 'price':price, 'seats_available':availability}
    
    df1 = pd.DataFrame(dict)
    #df.append(dict, ignore_index=True)
    df = pd.concat([df, df1], ignore_index=True)



print(df)    

                         route_name  \
0            Bangalore to Kozhikode   
1            Bangalore to Kozhikode   
2            Bangalore to Kozhikode   
3            Bangalore to Kozhikode   
4            Bangalore to Kozhikode   
..                              ...   
236  Kalpetta (kerala) to Kozhikode   
237  Kalpetta (kerala) to Kozhikode   
238  Kalpetta (kerala) to Kozhikode   
239  Kalpetta (kerala) to Kozhikode   
240  Kalpetta (kerala) to Kozhikode   

                                            route_link                busname  \
0    https://www.redbus.in/bus-tickets/bangalore-to...  KSRTC (Kerala) - 1586   
1    https://www.redbus.in/bus-tickets/bangalore-to...  KSRTC (Kerala) - 2133   
2    https://www.redbus.in/bus-tickets/bangalore-to...   KSRTC (Kerala) - 574   
3    https://www.redbus.in/bus-tickets/bangalore-to...   KSRTC (Kerala) - 583   
4    https://www.redbus.in/bus-tickets/bangalore-to...    KSRTC (Kerala) - 56   
..                                           

In [4]:
df.to_csv('df_kerala.csv', index=False)

# 2. RAJASTHAN

In [5]:
# experimental code (Rajasthan RSRTC)
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import mysql.connector

driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/rtc-directory")
driver.maximize_window()
rajasthan = driver.find_element(By.CSS_SELECTOR, 'a[href="/online-booking/rsrtc"]')
driver.execute_script("arguments[0].click();", rajasthan) 
#driver.find_element(By.LINK_TEXT, 'RSRTC').click()
# rajasthan.click();
time.sleep(3)
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")

rajasthan_route_names = []
rajasthan_route_links = []

def scrape_data():
    route_names = driver.find_elements(By.CLASS_NAME, "route")
    for route_name in route_names:
        rajasthan_route_names.append(route_name.text)

    route_links = driver.find_elements(By.CLASS_NAME, "route")
    hrefs = [route_link.get_attribute('href') for route_link in route_links]
    for href in hrefs:
        rajasthan_route_links.append(href)

page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")
for i in range(len(page_tabs)):
    if i > 0:
        page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")[i]
        driver.execute_script("arguments[0].click();", page_tabs)        
        time.sleep(4)
    scrape_data()




count = 0
for link in rajasthan_route_links:
    driver.get(link)
    driver.maximize_window()

    try:
        wait = WebDriverWait(driver, 3)
        wait_button = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "button")))
        driver.execute_script("arguments[0].click();", wait_button)
        # government_buses = driver.find_element(By.CLASS_NAME, "button")
        #driver.execute_script("arguments[0].click();", government_buses)
        
    except NoSuchElementException:
        pass
    except TimeoutException:
        pass

    # government_buses = driver.find_element(By.CLASS_NAME, "button")
    # driver.execute_script("arguments[0].click();", government_buses)
    # time.sleep(1)

    for t in range(33):     #scrolling 12 times so that whole website loads
        driver.execute_script("window.scrollBy(0, 650);")
        time.sleep(0.25)

    
    route_name = []
    route_link = []
    bus_name = []
    bus_type = []
    departing = []
    duration = []
    reaching = []
    rating = []
    price = []
    availability = []
    dict = {}
    df1 = pd.DataFrame()

    names = driver.find_elements(By.CLASS_NAME,"travels")
    types = driver.find_elements(By.CLASS_NAME,"bus-type")
    depart_timings = driver.find_elements(By.CLASS_NAME,"dp-time")
    travel_durations = driver.find_elements(By.CLASS_NAME,"dur")
    arrivals = driver.find_elements(By.CLASS_NAME,"bp-time")
    star_ratings = driver.find_elements(By.CLASS_NAME,"column-six")
    fares = driver.find_elements(By.CLASS_NAME,"fare")
    seats_available = driver.find_elements(By.CLASS_NAME,"seat-left")

    for name in names:
        bus_name.append(name.text)

    for type in types:
        bus_type.append(type.text)

    for depart_timing in depart_timings:
        departing.append(depart_timing.text)

    for travel_duration in travel_durations:
        duration.append(travel_duration.text)

    for arrival in arrivals:
        reaching.append(arrival.text)

    for star_rating in star_ratings:
        rating.append(star_rating.text[:3])

    for fare in fares:
        if (fare.text).isdigit():
            price.append(fare.text)
        else:
            price.append((fare.text).replace('INR','').strip())

    for seat_available in seats_available:
        availability.append(seat_available.text[:2].strip())
    
    for i in range(len(bus_name)):
        route_name.append(rajasthan_route_names[count])
        route_link.append(link)

    count += 1




    for i in range(len(rating)):
        if rating[i] == 'New' or rating[i] == ' ':
            rating[i] = '0.0'

    dict = {'route_name':route_name, 'route_link':route_link, 'busname':bus_name, 'bustype':bus_type, 'departing_time':departing,
            'duration':duration, 'reaching_time':reaching, 'star_rating':rating, 'price':price, 'seats_available':availability}
    
    df1 = pd.DataFrame(dict)
    df = pd.concat([df, df1], ignore_index=True)


print(df)    

                 route_name  \
0    Bangalore to Kozhikode   
1    Bangalore to Kozhikode   
2    Bangalore to Kozhikode   
3    Bangalore to Kozhikode   
4    Bangalore to Kozhikode   
..                      ...   
827        Bikaner to Sikar   
828        Bikaner to Sikar   
829        Bikaner to Sikar   
830        Bikaner to Sikar   
831        Bikaner to Sikar   

                                            route_link  \
0    https://www.redbus.in/bus-tickets/bangalore-to...   
1    https://www.redbus.in/bus-tickets/bangalore-to...   
2    https://www.redbus.in/bus-tickets/bangalore-to...   
3    https://www.redbus.in/bus-tickets/bangalore-to...   
4    https://www.redbus.in/bus-tickets/bangalore-to...   
..                                                 ...   
827  https://www.redbus.in/bus-tickets/bikaner-to-s...   
828  https://www.redbus.in/bus-tickets/bikaner-to-s...   
829  https://www.redbus.in/bus-tickets/bikaner-to-s...   
830  https://www.redbus.in/bus-tickets/bikaner-

# 3. HIMACHAL

In [6]:
# experimental code (Himachal HRTC)
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import mysql.connector

driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/rtc-directory")
driver.maximize_window()
himachal = driver.find_element(By.CSS_SELECTOR, 'a[href="/online-booking/hrtc"]')
driver.execute_script("arguments[0].click();", himachal) 
time.sleep(2)
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")

himachal_route_names = []
himachal_route_links = []

def scrape_data():
    route_names = driver.find_elements(By.CLASS_NAME, "route")
    for route_name in route_names:
        himachal_route_names.append(route_name.text)

    route_links = driver.find_elements(By.CLASS_NAME, "route")
    hrefs = [route_link.get_attribute('href') for route_link in route_links]
    for href in hrefs:
        himachal_route_links.append(href)

page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")
for i in range(len(page_tabs)):
    if i > 0:
        page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")[i]
        driver.execute_script("arguments[0].click();", page_tabs)        
        time.sleep(1)
    scrape_data()




count = 0
for link in himachal_route_links:
    driver.get(link)
    driver.maximize_window()

    try:
        wait = WebDriverWait(driver, 3)
        wait_button = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "button")))
        driver.execute_script("arguments[0].click();", wait_button)
        # government_buses = driver.find_element(By.CLASS_NAME, "button")
        #driver.execute_script("arguments[0].click();", government_buses)
        
    except NoSuchElementException:
        pass
    except TimeoutException:
        pass

    # government_buses = driver.find_element(By.CLASS_NAME, "p-left-10")
    # driver.execute_script("arguments[0].click();", government_buses)
    # time.sleep(1)

    for t in range(33):     #scrolling 12 times so that whole website loads
        driver.execute_script("window.scrollBy(0, 650);")
        time.sleep(0.25)

    
    route_name = []
    route_link = []
    bus_name = []
    bus_type = []
    departing = []
    duration = []
    reaching = []
    rating = []
    price = []
    availability = []
    dict = {}
    df1 = pd.DataFrame()

    names = driver.find_elements(By.CLASS_NAME,"travels")
    types = driver.find_elements(By.CLASS_NAME,"bus-type")
    depart_timings = driver.find_elements(By.CLASS_NAME,"dp-time")
    travel_durations = driver.find_elements(By.CLASS_NAME,"dur")
    arrivals = driver.find_elements(By.CLASS_NAME,"bp-time")
    star_ratings = driver.find_elements(By.CLASS_NAME,"column-six")
    fares = driver.find_elements(By.CLASS_NAME,"fare")
    seats_available = driver.find_elements(By.CLASS_NAME,"seat-left")

    for name in names:
        bus_name.append(name.text)

    for type in types:
        bus_type.append(type.text)

    for depart_timing in depart_timings:
        departing.append(depart_timing.text)

    for travel_duration in travel_durations:
        duration.append(travel_duration.text)

    for arrival in arrivals:
        reaching.append(arrival.text)

    for star_rating in star_ratings:
        rating.append(star_rating.text[:3])

    for fare in fares:
        if (fare.text).isdigit():
            price.append(fare.text)
        else:
            price.append((fare.text).replace('INR','').strip())

    for seat_available in seats_available:
        availability.append(seat_available.text[:2].strip())
    
    for i in range(len(bus_name)):
        route_name.append(himachal_route_names[count])
        route_link.append(link)

    count += 1



    for i in range(len(rating)):
        if rating[i] == 'New' or rating[i] == ' ':
            rating[i] = '0.0'

    dict = {'route_name':route_name, 'route_link':route_link, 'busname':bus_name, 'bustype':bus_type, 'departing_time':departing,
            'duration':duration, 'reaching_time':reaching, 'star_rating':rating, 'price':price, 'seats_available':availability}
    
    df1 = pd.DataFrame(dict)
    df = pd.concat([df, df1], ignore_index=True)


print(df)

                  route_name  \
0     Bangalore to Kozhikode   
1     Bangalore to Kozhikode   
2     Bangalore to Kozhikode   
3     Bangalore to Kozhikode   
4     Bangalore to Kozhikode   
...                      ...   
1399         Kangra to Delhi   
1400         Kangra to Delhi   
1401         Kangra to Delhi   
1402         Kangra to Delhi   
1403         Kangra to Delhi   

                                             route_link  \
0     https://www.redbus.in/bus-tickets/bangalore-to...   
1     https://www.redbus.in/bus-tickets/bangalore-to...   
2     https://www.redbus.in/bus-tickets/bangalore-to...   
3     https://www.redbus.in/bus-tickets/bangalore-to...   
4     https://www.redbus.in/bus-tickets/bangalore-to...   
...                                                 ...   
1399  https://www.redbus.in/bus-tickets/kangra-to-delhi   
1400  https://www.redbus.in/bus-tickets/kangra-to-delhi   
1401  https://www.redbus.in/bus-tickets/kangra-to-delhi   
1402  https://www.redbus.

# 4. PUNJAB

In [7]:

import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import mysql.connector

driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/rtc-directory")
driver.maximize_window()
punjab = driver.find_element(By.CSS_SELECTOR, 'a[href="/online-booking/pepsu-punjab"]')
driver.execute_script("arguments[0].click();", punjab) 
time.sleep(2)
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")

punjab_route_names = []
punjab_route_links = []

def scrape_data():
    route_names = driver.find_elements(By.CLASS_NAME, "route")
    for route_name in route_names:
        punjab_route_names.append(route_name.text)

    route_links = driver.find_elements(By.CLASS_NAME, "route")
    hrefs = [route_link.get_attribute('href') for route_link in route_links]
    for href in hrefs:
        punjab_route_links.append(href)

page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")
for i in range(len(page_tabs)):
    if i > 0:
        page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")[i]
        driver.execute_script("arguments[0].click();", page_tabs)        
        time.sleep(1)
    scrape_data()



count = 0
for link in punjab_route_links:
    driver.get(link)
    driver.maximize_window()

    try:
        wait = WebDriverWait(driver, 3)
        wait_button = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "button")))
        driver.execute_script("arguments[0].click();", wait_button)
        # government_buses = driver.find_element(By.CLASS_NAME, "button")
        #driver.execute_script("arguments[0].click();", government_buses)
        
    except NoSuchElementException:
        pass
    except TimeoutException:
        pass

    # government_buses = driver.find_element(By.CLASS_NAME, "p-left-10")
    # driver.execute_script("arguments[0].click();", government_buses)
    # time.sleep(1)

    for t in range(32):     #scrolling 12 times so that whole website loads
        driver.execute_script("window.scrollBy(0, 650);")
        time.sleep(0.25)

    
    route_name = []
    route_link = []
    bus_name = []
    bus_type = []
    departing = []
    duration = []
    reaching = []
    rating = []
    price = []
    availability = []
    dict = {}
    df1 = pd.DataFrame()

    names = driver.find_elements(By.CLASS_NAME,"travels")
    types = driver.find_elements(By.CLASS_NAME,"bus-type")
    depart_timings = driver.find_elements(By.CLASS_NAME,"dp-time")
    travel_durations = driver.find_elements(By.CLASS_NAME,"dur")
    arrivals = driver.find_elements(By.CLASS_NAME,"bp-time")
    star_ratings = driver.find_elements(By.CLASS_NAME,"column-six")
    fares = driver.find_elements(By.CLASS_NAME,"fare")
    seats_available = driver.find_elements(By.CLASS_NAME,"seat-left")

    for name in names:
        bus_name.append(name.text)

    for type in types:
        bus_type.append(type.text)

    for depart_timing in depart_timings:
        departing.append(depart_timing.text)

    for travel_duration in travel_durations:
        duration.append(travel_duration.text)

    for arrival in arrivals:
        reaching.append(arrival.text)

    for star_rating in star_ratings:
        rating.append(star_rating.text[:3])

    for fare in fares:
        if (fare.text).isdigit():
            price.append(fare.text)
        else:
            price.append((fare.text).replace('INR','').strip())

    for seat_available in seats_available:
        availability.append(seat_available.text[:2].strip())
    
    for i in range(len(bus_name)):
        route_name.append(punjab_route_names[count])
        route_link.append(link)

    count += 1





    for i in range(len(rating)):
        if rating[i] == 'New' or rating[i] == ' ':
            rating[i] = '0.0'

    dict = {'route_name':route_name, 'route_link':route_link, 'busname':bus_name, 'bustype':bus_type, 'departing_time':departing,
            'duration':duration, 'reaching_time':reaching, 'star_rating':rating, 'price':price, 'seats_available':availability}
    
    df1 = pd.DataFrame(dict)
    df = pd.concat([df, df1], ignore_index=True)


print(df)    

                  route_name  \
0     Bangalore to Kozhikode   
1     Bangalore to Kozhikode   
2     Bangalore to Kozhikode   
3     Bangalore to Kozhikode   
4     Bangalore to Kozhikode   
...                      ...   
1756  Chandigarh to Bathinda   
1757  Chandigarh to Bathinda   
1758  Chandigarh to Bathinda   
1759  Chandigarh to Bathinda   
1760  Chandigarh to Bathinda   

                                             route_link  \
0     https://www.redbus.in/bus-tickets/bangalore-to...   
1     https://www.redbus.in/bus-tickets/bangalore-to...   
2     https://www.redbus.in/bus-tickets/bangalore-to...   
3     https://www.redbus.in/bus-tickets/bangalore-to...   
4     https://www.redbus.in/bus-tickets/bangalore-to...   
...                                                 ...   
1756  https://www.redbus.in/bus-tickets/chandigarh-t...   
1757  https://www.redbus.in/bus-tickets/chandigarh-t...   
1758  https://www.redbus.in/bus-tickets/chandigarh-t...   
1759  https://www.redbus.

# 5. BIHAR

In [8]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import mysql.connector

driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/rtc-directory")
driver.maximize_window()
bihar = driver.find_element(By.CSS_SELECTOR, 'a[href="/online-booking/bihar-state-road-transport-corporation-bsrtc"]')
driver.execute_script("arguments[0].click();", bihar) 
time.sleep(2)
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")

bihar_route_names = []
bihar_route_links = []

def scrape_data():
    route_names = driver.find_elements(By.CLASS_NAME, "route")
    for route_name in route_names:
        bihar_route_names.append(route_name.text)

    route_links = driver.find_elements(By.CLASS_NAME, "route")
    hrefs = [route_link.get_attribute('href') for route_link in route_links]
    for href in hrefs:
        bihar_route_links.append(href)

page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")
for i in range(len(page_tabs)):
    if i > 0:
        page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")[i]
        driver.execute_script("arguments[0].click();", page_tabs)        
        time.sleep(1)
    scrape_data()



count = 0
for link in bihar_route_links:
    driver.get(link)
    driver.maximize_window()

    try:
        wait = WebDriverWait(driver, 3)
        wait_button = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "button")))
        driver.execute_script("arguments[0].click();", wait_button)
        # government_buses = driver.find_element(By.CLASS_NAME, "button")
        #driver.execute_script("arguments[0].click();", government_buses)
        
    except NoSuchElementException:
        pass
    except TimeoutException:
        pass

    # government_buses = driver.find_element(By.CLASS_NAME, "p-left-10")
    # driver.execute_script("arguments[0].click();", government_buses)
    # time.sleep(1)

    for t in range(30):     #scrolling 12 times so that whole website loads
        driver.execute_script("window.scrollBy(0, 650);")
        time.sleep(0.25)

    
    route_name = []
    route_link = []
    bus_name = []
    bus_type = []
    departing = []
    duration = []
    reaching = []
    rating = []
    price = []
    availability = []
    dict = {}
    df1 = pd.DataFrame()

    names = driver.find_elements(By.CLASS_NAME,"travels")
    types = driver.find_elements(By.CLASS_NAME,"bus-type")
    depart_timings = driver.find_elements(By.CLASS_NAME,"dp-time")
    travel_durations = driver.find_elements(By.CLASS_NAME,"dur")
    arrivals = driver.find_elements(By.CLASS_NAME,"bp-time")
    star_ratings = driver.find_elements(By.CLASS_NAME,"column-six")
    fares = driver.find_elements(By.CLASS_NAME,"fare")
    seats_available = driver.find_elements(By.CLASS_NAME,"seat-left")

    for name in names:
        bus_name.append(name.text)

    for type in types:
        bus_type.append(type.text)

    for depart_timing in depart_timings:
        departing.append(depart_timing.text)

    for travel_duration in travel_durations:
        duration.append(travel_duration.text)

    for arrival in arrivals:
        reaching.append(arrival.text)

    for star_rating in star_ratings:
        rating.append(star_rating.text[:3])

    for fare in fares:
        if (fare.text).isdigit():
            price.append(fare.text)
        else:
            price.append((fare.text).replace('INR','').strip())

    for seat_available in seats_available:
        availability.append(seat_available.text[:2].strip())
    
    for i in range(len(bus_name)):
        route_name.append(bihar_route_names[count])
        route_link.append(link)

    count += 1




    for i in range(len(rating)):
        if rating[i] == 'New' or rating[i] == ' ':
            rating[i] = '0.0'

    dict = {'route_name':route_name, 'route_link':route_link, 'busname':bus_name, 'bustype':bus_type, 'departing_time':departing,
            'duration':duration, 'reaching_time':reaching, 'star_rating':rating, 'price':price, 'seats_available':availability}
    
    df1 = pd.DataFrame(dict)
    df = pd.concat([df, df1], ignore_index=True)



print(df)   

                        route_name  \
0           Bangalore to Kozhikode   
1           Bangalore to Kozhikode   
2           Bangalore to Kozhikode   
3           Bangalore to Kozhikode   
4           Bangalore to Kozhikode   
...                            ...   
1893  Lucknow to Gopalganj (Bihar)   
1894  Lucknow to Gopalganj (Bihar)   
1895  Lucknow to Gopalganj (Bihar)   
1896  Lucknow to Gopalganj (Bihar)   
1897  Lucknow to Gopalganj (Bihar)   

                                             route_link  \
0     https://www.redbus.in/bus-tickets/bangalore-to...   
1     https://www.redbus.in/bus-tickets/bangalore-to...   
2     https://www.redbus.in/bus-tickets/bangalore-to...   
3     https://www.redbus.in/bus-tickets/bangalore-to...   
4     https://www.redbus.in/bus-tickets/bangalore-to...   
...                                                 ...   
1893  https://www.redbus.in/bus-tickets/lucknow-to-g...   
1894  https://www.redbus.in/bus-tickets/lucknow-to-g...   
1895  https:

# 6. SOUTH BENGAL

In [9]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import mysql.connector

driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/rtc-directory")
driver.maximize_window()
s_bengal = driver.find_element(By.CSS_SELECTOR, 'a[href="/online-booking/south-bengal-state-transport-corporation-sbstc"]')
driver.execute_script("arguments[0].click();", s_bengal) 
time.sleep(2)
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")

s_bengal_route_names = []
s_bengal_route_links = []

def scrape_data():
    route_names = driver.find_elements(By.CLASS_NAME, "route")
    for route_name in route_names:
        s_bengal_route_names.append(route_name.text)

    route_links = driver.find_elements(By.CLASS_NAME, "route")
    hrefs = [route_link.get_attribute('href') for route_link in route_links]
    for href in hrefs:
        s_bengal_route_links.append(href)

page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")
for i in range(len(page_tabs)):
    if i > 0:
        page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")[i]
        driver.execute_script("arguments[0].click();", page_tabs)        
        time.sleep(1)
    scrape_data()

# print(s_bengal_route_names)
# print(s_bengal_route_links)


count = 0
for link in s_bengal_route_links:
    driver.get(link)
    driver.maximize_window()

    try:
        wait = WebDriverWait(driver, 3)
        wait_button = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "button")))
        driver.execute_script("arguments[0].click();", wait_button)
        # government_buses = driver.find_element(By.CLASS_NAME, "button")
        #driver.execute_script("arguments[0].click();", government_buses)
        
    except NoSuchElementException:
        pass
    except TimeoutException:
        pass

    # government_buses = driver.find_element(By.CLASS_NAME, "p-left-10")
    # driver.execute_script("arguments[0].click();", government_buses)
    # time.sleep(1)

    for t in range(25):     #scrolling 12 times so that whole website loads
        driver.execute_script("window.scrollBy(0, 650);")
        time.sleep(0.25)

    
    route_name = []
    route_link = []
    bus_name = []
    bus_type = []
    departing = []
    duration = []
    reaching = []
    rating = []
    price = []
    availability = []
    dict ={}
    df1 = pd.DataFrame()

    names = driver.find_elements(By.CLASS_NAME,"travels")
    types = driver.find_elements(By.CLASS_NAME,"bus-type")
    depart_timings = driver.find_elements(By.CLASS_NAME,"dp-time")
    travel_durations = driver.find_elements(By.CLASS_NAME,"dur")
    arrivals = driver.find_elements(By.CLASS_NAME,"bp-time")
    star_ratings = driver.find_elements(By.CLASS_NAME,"column-six")
    fares = driver.find_elements(By.CLASS_NAME,"fare")
    seats_available = driver.find_elements(By.CLASS_NAME,"seat-left")

    for name in names:
        bus_name.append(name.text)

    for type in types:
        bus_type.append(type.text)

    for depart_timing in depart_timings:
        departing.append(depart_timing.text)

    for travel_duration in travel_durations:
        duration.append(travel_duration.text)

    for arrival in arrivals:
        reaching.append(arrival.text)

    for star_rating in star_ratings:
        rating.append(star_rating.text[:3])

    for fare in fares:
        if (fare.text).isdigit():
            price.append(fare.text)
        else:
            price.append((fare.text).replace('INR','').strip())

    for seat_available in seats_available:
        availability.append(seat_available.text[:2].strip())
    
    for i in range(len(bus_name)):
        route_name.append(s_bengal_route_names[count])
        route_link.append(link)

    count += 1




    for i in range(len(rating)):
        if rating[i] == 'New' or rating[i] == ' ':
            rating[i] = '0.0'

    dict = {'route_name':route_name, 'route_link':route_link, 'busname':bus_name, 'bustype':bus_type, 'departing_time':departing,
            'duration':duration, 'reaching_time':reaching, 'star_rating':rating, 'price':price, 'seats_available':availability}
    
    df1 = pd.DataFrame(dict)
    df = pd.concat([df, df1], ignore_index=True)

  
  
print(df)    

                             route_name  \
0                Bangalore to Kozhikode   
1                Bangalore to Kozhikode   
2                Bangalore to Kozhikode   
3                Bangalore to Kozhikode   
4                Bangalore to Kozhikode   
...                                 ...   
2151  Purulia to Durgapur (West Bengal)   
2152  Purulia to Durgapur (West Bengal)   
2153  Purulia to Durgapur (West Bengal)   
2154  Purulia to Durgapur (West Bengal)   
2155  Purulia to Durgapur (West Bengal)   

                                             route_link  \
0     https://www.redbus.in/bus-tickets/bangalore-to...   
1     https://www.redbus.in/bus-tickets/bangalore-to...   
2     https://www.redbus.in/bus-tickets/bangalore-to...   
3     https://www.redbus.in/bus-tickets/bangalore-to...   
4     https://www.redbus.in/bus-tickets/bangalore-to...   
...                                                 ...   
2151  https://www.redbus.in/bus-tickets/purulia-to-d...   
2152  https

# 7. WEST BENGAL

In [10]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import mysql.connector

driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/rtc-directory")
driver.maximize_window()
w_bengal = driver.find_element(By.CSS_SELECTOR, 'a[href="/online-booking/west-bengal-transport-corporation"]')
driver.execute_script("arguments[0].click();", w_bengal) 
time.sleep(2)
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")

w_bengal_route_names = []
w_bengal_route_links = []

def scrape_data():
    route_names = driver.find_elements(By.CLASS_NAME, "route")
    for route_name in route_names:
        w_bengal_route_names.append(route_name.text)

    route_links = driver.find_elements(By.CLASS_NAME, "route")
    hrefs = [route_link.get_attribute('href') for route_link in route_links]
    for href in hrefs:
        w_bengal_route_links.append(href)

# page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")
# for i in range(len(page_tabs)):
#     if i > 0:
#         page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")[i]
#         driver.execute_script("arguments[0].click();", page_tabs)        
#         time.sleep(1)
#     scrape_data()

scrape_data()




count = 0
for link in w_bengal_route_links:
    driver.get(link)
    driver.maximize_window()

    # government_buses = driver.find_element(By.CLASS_NAME, "p-left-10")
    # driver.execute_script("arguments[0].click();", government_buses)
    # time.sleep(1)

    for t in range(20):     #scrolling 12 times so that whole website loads
        driver.execute_script("window.scrollBy(0, 500);")
        time.sleep(0.3)

    
    route_name = []
    route_link = []
    bus_name = []
    bus_type = []
    departing = []
    duration = []
    reaching = []
    rating = []
    price = []
    availability = []
    dict = {}
    df1 = pd.DataFrame()

    names = driver.find_elements(By.CLASS_NAME,"travels")
    types = driver.find_elements(By.CLASS_NAME,"bus-type")
    depart_timings = driver.find_elements(By.CLASS_NAME,"dp-time")
    travel_durations = driver.find_elements(By.CLASS_NAME,"dur")
    arrivals = driver.find_elements(By.CLASS_NAME,"bp-time")
    star_ratings = driver.find_elements(By.CLASS_NAME,"column-six")
    fares = driver.find_elements(By.CLASS_NAME,"fare")
    seats_available = driver.find_elements(By.CLASS_NAME,"seat-left")

    for name in names:
        bus_name.append(name.text)

    for type in types:
        bus_type.append(type.text)

    for depart_timing in depart_timings:
        departing.append(depart_timing.text)

    for travel_duration in travel_durations:
        duration.append(travel_duration.text)

    for arrival in arrivals:
        reaching.append(arrival.text)

    for star_rating in star_ratings:
        rating.append(star_rating.text[:3])

    for fare in fares:
        if (fare.text).isdigit():
            price.append(fare.text)
        else:
            price.append((fare.text).replace('INR','').strip())

    for seat_available in seats_available:
        availability.append(seat_available.text[:2].strip())
    
    for i in range(len(bus_name)):
        route_name.append(w_bengal_route_names[count])
        route_link.append(link)

    count += 1



    for i in range(len(rating)):
        if rating[i] == 'New' or rating[i] == ' ':
            rating[i] = '0.0'

    dict = {'route_name':route_name, 'route_link':route_link, 'busname':bus_name, 'bustype':bus_type, 'departing_time':departing,
            'duration':duration, 'reaching_time':reaching, 'star_rating':rating, 'price':price, 'seats_available':availability}
    
    df1 = pd.DataFrame(dict)
    df = pd.concat([df, df1], ignore_index=True)


print(df)    

                  route_name  \
0     Bangalore to Kozhikode   
1     Bangalore to Kozhikode   
2     Bangalore to Kozhikode   
3     Bangalore to Kozhikode   
4     Bangalore to Kozhikode   
...                      ...   
2363   Kolkata to Mandarmani   
2364   Kolkata to Mandarmani   
2365   Kolkata to Mandarmani   
2366     Kolkata to Bakkhali   
2367     Kolkata to Bakkhali   

                                             route_link  \
0     https://www.redbus.in/bus-tickets/bangalore-to...   
1     https://www.redbus.in/bus-tickets/bangalore-to...   
2     https://www.redbus.in/bus-tickets/bangalore-to...   
3     https://www.redbus.in/bus-tickets/bangalore-to...   
4     https://www.redbus.in/bus-tickets/bangalore-to...   
...                                                 ...   
2363  https://www.redbus.in/bus-tickets/kolkata-to-m...   
2364  https://www.redbus.in/bus-tickets/kolkata-to-m...   
2365  https://www.redbus.in/bus-tickets/kolkata-to-m...   
2366  https://www.redbus.

# 8. CHANDIGARH 

In [11]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import mysql.connector

driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/rtc-directory")
driver.maximize_window()
chandigarh = driver.find_element(By.CSS_SELECTOR, 'a[href="/online-booking/chandigarh-transport-undertaking-ctu"]')
driver.execute_script("arguments[0].click();", chandigarh) 
time.sleep(2)
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")

chandigarh_route_names = []
chandigarh_route_links = []

def scrape_data():
    route_names = driver.find_elements(By.CLASS_NAME, "route")
    for route_name in route_names:
        chandigarh_route_names.append(route_name.text)

    route_links = driver.find_elements(By.CLASS_NAME, "route")
    hrefs = [route_link.get_attribute('href') for route_link in route_links]
    for href in hrefs:
        chandigarh_route_links.append(href)

page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")
for i in range(len(page_tabs)):
    if i > 0:
        page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")[i]
        driver.execute_script("arguments[0].click();", page_tabs)        
        time.sleep(1)
    scrape_data()





count = 0
for link in chandigarh_route_links:
    driver.get(link)
    driver.maximize_window()

    try:
        wait = WebDriverWait(driver, 3)
        wait_button = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "button")))
        driver.execute_script("arguments[0].click();", wait_button)
        # government_buses = driver.find_element(By.CLASS_NAME, "button")
        #driver.execute_script("arguments[0].click();", government_buses)
        
    except NoSuchElementException:
        pass
    except TimeoutException:
        pass

    

    for t in range(20):     #scrolling 12 times so that whole website loads
        driver.execute_script("window.scrollBy(0, 500);")
        time.sleep(0.3)

    
    route_name = []
    route_link = []
    bus_name = []
    bus_type = []
    departing = []
    duration = []
    reaching = []
    rating = []
    price = []
    availability = []
    dict = {}
    df1 = pd.DataFrame()

    names = driver.find_elements(By.CLASS_NAME,"travels")
    types = driver.find_elements(By.CLASS_NAME,"bus-type")
    depart_timings = driver.find_elements(By.CLASS_NAME,"dp-time")
    travel_durations = driver.find_elements(By.CLASS_NAME,"dur")
    arrivals = driver.find_elements(By.CLASS_NAME,"bp-time")
    star_ratings = driver.find_elements(By.CLASS_NAME,"column-six")
    fares = driver.find_elements(By.CLASS_NAME,"fare")
    seats_available = driver.find_elements(By.CLASS_NAME,"seat-left")

    for name in names:
        bus_name.append(name.text)

    for type in types:
        bus_type.append(type.text)

    for depart_timing in depart_timings:
        departing.append(depart_timing.text)

    for travel_duration in travel_durations:
        duration.append(travel_duration.text)

    for arrival in arrivals:
        reaching.append(arrival.text)

    for star_rating in star_ratings:
        rating.append(star_rating.text[:3])

    for fare in fares:
        if (fare.text).isdigit():
            price.append(fare.text)
        else:
            price.append((fare.text).replace('INR','').strip())

    for seat_available in seats_available:
        availability.append(seat_available.text[:2].strip())
    
    for i in range(len(bus_name)):
        route_name.append(chandigarh_route_names[count])
        route_link.append(link)

    count += 1


    # print(route_name)
    # print(route_link)
    # print(bus_name)
    # print(bus_type)
    # print(departing)
    # print(duration)
    # print(reaching)
    # print(rating)
    # print(price)
    # print(availability)

    for i in range(len(rating)):
        if rating[i] == 'New' or rating[i] == ' ':
            rating[i] = '0.0'

    dict = {'route_name':route_name, 'route_link':route_link, 'busname':bus_name, 'bustype':bus_type, 'departing_time':departing,
            'duration':duration, 'reaching_time':reaching, 'star_rating':rating, 'price':price, 'seats_available':availability}
    
    df1 = pd.DataFrame(dict)
    df = pd.concat([df, df1], ignore_index=True)

print(df)    

                   route_name  \
0      Bangalore to Kozhikode   
1      Bangalore to Kozhikode   
2      Bangalore to Kozhikode   
3      Bangalore to Kozhikode   
4      Bangalore to Kozhikode   
...                       ...   
3656  Chandigarh to Jawala Ji   
3657  Chandigarh to Jawala Ji   
3658  Chandigarh to Jawala Ji   
3659  Chandigarh to Jawala Ji   
3660  Chandigarh to Jawala Ji   

                                             route_link  \
0     https://www.redbus.in/bus-tickets/bangalore-to...   
1     https://www.redbus.in/bus-tickets/bangalore-to...   
2     https://www.redbus.in/bus-tickets/bangalore-to...   
3     https://www.redbus.in/bus-tickets/bangalore-to...   
4     https://www.redbus.in/bus-tickets/bangalore-to...   
...                                                 ...   
3656  https://www.redbus.in/bus-tickets/chandigarh-t...   
3657  https://www.redbus.in/bus-tickets/chandigarh-t...   
3658  https://www.redbus.in/bus-tickets/chandigarh-t...   
3659  https:/

# 9. ASSAM

In [12]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import mysql.connector

driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/rtc-directory")
driver.maximize_window()
assam = driver.find_element(By.CSS_SELECTOR, 'a[href="/online-booking/astc"]')
driver.execute_script("arguments[0].click();", assam) 
time.sleep(2)
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")

assam_route_names = []
assam_route_links = []

def scrape_data():
    route_names = driver.find_elements(By.CLASS_NAME, "route")
    for route_name in route_names:
        assam_route_names.append(route_name.text)

    route_links = driver.find_elements(By.CLASS_NAME, "route")
    hrefs = [route_link.get_attribute('href') for route_link in route_links]
    for href in hrefs:
        assam_route_links.append(href)

page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")
for i in range(len(page_tabs)):
    if i > 0:
        page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")[i]
        driver.execute_script("arguments[0].click();", page_tabs)        
        time.sleep(1)
    scrape_data()


# print(assam_route_names)
# print(assam_route_links)


count = 0
for link in assam_route_links:
    driver.get(link)
    driver.maximize_window()

    try:
        wait = WebDriverWait(driver, 3)
        wait_button = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "button")))
        driver.execute_script("arguments[0].click();", wait_button)
        # government_buses = driver.find_element(By.CLASS_NAME, "button")
        #driver.execute_script("arguments[0].click();", government_buses)
        
    except NoSuchElementException:
        pass
    except TimeoutException:
        pass

    # government_buses = driver.find_element(By.CLASS_NAME, "p-left-10")
    # driver.execute_script("arguments[0].click();", government_buses)
    # time.sleep(1)

    for t in range(20):     #scrolling 12 times so that whole website loads
        driver.execute_script("window.scrollBy(0, 500);")
        time.sleep(0.3)

    
    route_name = []
    route_link = []
    bus_name = []
    bus_type = []
    departing = []
    duration = []
    reaching = []
    rating = []
    price = []
    availability = []
    dict = {}
    df1 = pd.DataFrame()

    names = driver.find_elements(By.CLASS_NAME,"travels")
    types = driver.find_elements(By.CLASS_NAME,"bus-type")
    depart_timings = driver.find_elements(By.CLASS_NAME,"dp-time")
    travel_durations = driver.find_elements(By.CLASS_NAME,"dur")
    arrivals = driver.find_elements(By.CLASS_NAME,"bp-time")
    star_ratings = driver.find_elements(By.CLASS_NAME,"column-six")
    fares = driver.find_elements(By.CLASS_NAME,"fare")
    seats_available = driver.find_elements(By.CLASS_NAME,"seat-left")

    for name in names:
        bus_name.append(name.text)

    for type in types:
        bus_type.append(type.text)

    for depart_timing in depart_timings:
        departing.append(depart_timing.text)

    for travel_duration in travel_durations:
        duration.append(travel_duration.text)

    for arrival in arrivals:
        reaching.append(arrival.text)

    for star_rating in star_ratings:
        rating.append(star_rating.text[:3])

    for fare in fares:
        if (fare.text).isdigit():
            price.append(fare.text)
        else:
            price.append((fare.text).replace('INR','').strip())

    for seat_available in seats_available:
        availability.append(seat_available.text[:2].strip())
    
    for i in range(len(bus_name)):
        route_name.append(assam_route_names[count])
        route_link.append(link)

    count += 1



    for i in range(len(rating)):
        if rating[i] == 'New' or rating[i] == ' ':
            rating[i] = '0.0'

    dict = {'route_name':route_name, 'route_link':route_link, 'busname':bus_name, 'bustype':bus_type, 'departing_time':departing,
            'duration':duration, 'reaching_time':reaching, 'star_rating':rating, 'price':price, 'seats_available':availability}
    
    df1 = pd.DataFrame(dict)
    df = pd.concat([df, df1], ignore_index=True)


print(df)

                  route_name  \
0     Bangalore to Kozhikode   
1     Bangalore to Kozhikode   
2     Bangalore to Kozhikode   
3     Bangalore to Kozhikode   
4     Bangalore to Kozhikode   
...                      ...   
4069   Dibrugarh to Bokakhat   
4070   Dibrugarh to Bokakhat   
4071   Dibrugarh to Bokakhat   
4072   Dibrugarh to Bokakhat   
4073   Dibrugarh to Bokakhat   

                                             route_link  \
0     https://www.redbus.in/bus-tickets/bangalore-to...   
1     https://www.redbus.in/bus-tickets/bangalore-to...   
2     https://www.redbus.in/bus-tickets/bangalore-to...   
3     https://www.redbus.in/bus-tickets/bangalore-to...   
4     https://www.redbus.in/bus-tickets/bangalore-to...   
...                                                 ...   
4069  https://www.redbus.in/bus-tickets/dibrugarh-to...   
4070  https://www.redbus.in/bus-tickets/dibrugarh-to...   
4071  https://www.redbus.in/bus-tickets/dibrugarh-to...   
4072  https://www.redbus.

# 10. KADAMBA

In [13]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import mysql.connector

driver = webdriver.Chrome()
driver.get("https://www.redbus.in/online-booking/rtc-directory")
driver.maximize_window()
kadamba = driver.find_element(By.CSS_SELECTOR, 'a[href="/online-booking/ktcl"]')
#kadamba.click();
driver.execute_script("arguments[0].click();", kadamba) 
time.sleep(3)
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")
driver.execute_script("window.scrollBy(0, window.innerHeight);")

kadamba_route_names = []
kadamba_route_links = []

def scrape_data():
    route_names = driver.find_elements(By.CLASS_NAME, "route")
    for route_name in route_names:
        kadamba_route_names.append(route_name.text)

    route_links = driver.find_elements(By.CLASS_NAME, "route")
    hrefs = [route_link.get_attribute('href') for route_link in route_links]
    for href in hrefs:
        kadamba_route_links.append(href)

page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")
for i in range(len(page_tabs)):
    if i > 0:
        page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs ")[i]
        driver.execute_script("arguments[0].click();", page_tabs)        
        time.sleep(2)
    scrape_data()




count = 0
for link in kadamba_route_links:
    driver.get(link)
    driver.maximize_window()
    

    for t in range(22):     #scrolling 16 times so that whole website loads
        driver.execute_script("window.scrollBy(0, 500);")
        time.sleep(0.5)

    
    route_name = []
    route_link = []
    bus_name = []
    bus_type = []
    departing = []
    duration = []
    reaching = []
    rating = []
    price = []
    availability = []
    dict = {}
    df1 = pd.DataFrame()

    names = driver.find_elements(By.CLASS_NAME,"travels")
    types = driver.find_elements(By.CLASS_NAME,"bus-type")
    depart_timings = driver.find_elements(By.CLASS_NAME,"dp-time")
    travel_durations = driver.find_elements(By.CLASS_NAME,"dur")
    arrivals = driver.find_elements(By.CLASS_NAME,"bp-time")
    star_ratings = driver.find_elements(By.CLASS_NAME,"column-six")
    fares = driver.find_elements(By.CLASS_NAME,"fare")
    seats_available = driver.find_elements(By.CLASS_NAME,"seat-left")

    for name in names:
        bus_name.append(name.text)

    for type in types:
        bus_type.append(type.text)

    for depart_timing in depart_timings:
        departing.append(depart_timing.text)

    for travel_duration in travel_durations:
        duration.append(travel_duration.text)

    for arrival in arrivals:
        reaching.append(arrival.text)

    for star_rating in star_ratings:
        rating.append(star_rating.text[:3])

    for fare in fares:
        if (fare.text).isdigit():
            price.append(fare.text)
        else:
            price.append((fare.text).replace('INR','').strip())

    for seat_available in seats_available:
        availability.append(seat_available.text[:2].strip())
    
    for i in range(len(bus_name)):
        route_name.append(kadamba_route_names[count])
        route_link.append(link)

    count += 1


 

    for i in range(len(rating)):
        if rating[i] == 'New' or rating[i] == ' ':
            rating[i] = '0.0'
    
    dict = {'route_name':route_name, 'route_link':route_link, 'busname':bus_name, 'bustype':bus_type, 'departing_time':departing,
            'duration':duration, 'reaching_time':reaching, 'star_rating':rating, 'price':price, 'seats_available':availability}
    
    df1 = pd.DataFrame(dict)
    df = pd.concat([df, df1], ignore_index=True)



print(df)   

                     route_name  \
0        Bangalore to Kozhikode   
1        Bangalore to Kozhikode   
2        Bangalore to Kozhikode   
3        Bangalore to Kozhikode   
4        Bangalore to Kozhikode   
...                         ...   
4552  Calangute (goa) to Panaji   
4553      Goa Airport to Panaji   
4554      Goa Airport to Panaji   
4555      Goa Airport to Panaji   
4556      Goa Airport to Panaji   

                                             route_link  \
0     https://www.redbus.in/bus-tickets/bangalore-to...   
1     https://www.redbus.in/bus-tickets/bangalore-to...   
2     https://www.redbus.in/bus-tickets/bangalore-to...   
3     https://www.redbus.in/bus-tickets/bangalore-to...   
4     https://www.redbus.in/bus-tickets/bangalore-to...   
...                                                 ...   
4552  https://www.redbus.in/bus-tickets/calangute-go...   
4553  https://www.redbus.in/bus-tickets/goa-airport-...   
4554  https://www.redbus.in/bus-tickets/goa-airp

In [17]:
df.to_csv('final_data1.csv')

In [18]:
df = pd.read_csv('final_data1.csv')
df = df.reset_index(drop=True)
print(df.dtypes)
df = df.astype({'seats_available': 'int'})
print(df.dtypes)

Unnamed: 0           int64
Unnamed: 0.1         int64
route_name          object
route_link          object
busname             object
bustype             object
departing_time      object
duration            object
reaching_time       object
star_rating        float64
price              float64
seats_available      int64
dtype: object
Unnamed: 0           int64
Unnamed: 0.1         int64
route_name          object
route_link          object
busname             object
bustype             object
departing_time      object
duration            object
reaching_time       object
star_rating        float64
price              float64
seats_available      int32
dtype: object


In [19]:
df

,Unnamed: 0,Unnamed: 0.1,route_name,route_link,busname,bustype,departing_time,duration,reaching_time,star_rating,price,seats_available
0,0,0,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 1586,Super Express Non AC Seater Air Bus (2+2),21:31,09h 05m,06:36,3.4,640.0,17
1,1,1,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 2133,Swift Deluxe Non AC Air Bus (2+2),22:12,09h 24m,07:36,3.8,640.0,10
2,2,2,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 574,Super Express Non AC Seater Air Bus (2+2),23:00,10h 30m,09:30,4.1,469.0,15
3,3,3,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 583,Super Express Non AC Seater Air Bus (2+2),23:30,10h 05m,09:35,3.2,469.0,11
4,4,4,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 56,SWIFT-GARUDA A/C SEATER BUS,23:41,09h 24m,09:05,3.5,627.0,21
...,...,...,...,...,...,...,...,...,...,...,...,...
4552,4552,4552,Calangute (goa) to Panaji,https://www.redbus.in/bus-tickets/calangute-go...,Kadamba Transport Corporation Limited (KTCL) -...,AC Seater (2+2),15:30,00h 30m,16:00,3.7,100.0,41
4553,4553,4553,Goa Airport to Panaji,https://www.redbus.in/bus-tickets/goa-airport-...,Kadamba Transport Corporation Limited (KTCL) -...,AC Seater (2+2),07:30,01h 00m,08:30,4.4,150.0,41
4554,4554,4554,Goa Airport to Panaji,https://www.redbus.in/bus-tickets/goa-airport-...,Kadamba Transport Corporation Limited (KTCL) -...,Volvo AC Seater 2+2,08:00,01h 00m,09:00,4.4,150.0,40
4555,4555,4555,Goa Airport to Panaji,https://www.redbus.in/bus-tickets/goa-airport-...,Kadamba Transport Corporation Limited (KTCL) -...,AC Seater (2+2),13:45,01h 00m,14:45,4.4,150.0,41


In [28]:
import pandas as pd
import mysql.connector

con = mysql.connector.connect(
    
host="localhost",
user="root",
password="Qdpfp7xsm2@"
    
)

cursor = con.cursor()

query = "create database if not exists BUS_DATA"
cursor.execute(query)

query = "use BUS_DATA"
cursor.execute(query)

query = """create table if not exists All_bus_data(id INT AUTO_INCREMENT PRIMARY KEY, route_name varchar(70), 
        route_link varchar(180), busname varchar(100), bustype varchar(50), departing_time TIME,
        duration varchar(20), reaching_time TIME, star_rating float(2,1), price DECIMAL(7, 2), seats_available INT)"""
cursor.execute(query)
for i in range(len(df)):
    data = (df.loc[i, 'route_name'], df.loc[i, 'route_link'], df.loc[i, 'busname'], df.loc[i, 'bustype'], df.loc[i, 'departing_time'], df.loc[i, 'duration'], 
            df.loc[i, 'reaching_time'], df.loc[i, 'star_rating'], df.loc[i, 'price'], df.loc[i, 'seats_available'])

query = """insert into All_bus_data(route_name, route_link, busname, bustype, departing_time, duration, reaching_time, star_rating, price, seats_available) 
        values ('{}','{}','{}','{}',{},'{}',{},{},{},{})""".format(data)

cursor.execute(query)
con.commit()

IndexError: Replacement index 1 out of range for positional args tuple

In [33]:
import pandas as pd
import mysql.connector
con = mysql.connector.connect(
    
host="localhost",
user="root",
password="Qdpfp7xsm2@"
    
)
cursor = con.cursor()

query = "create database if not exists BUS_DATA "
cursor.execute(query)

query = "use BUS_DATA"
cursor.execute(query)

query = """create table if not exists All_bus_data(id INT AUTO_INCREMENT PRIMARY KEY, route_name varchar(70), 
        route_link varchar(180), busname varchar(100), bustype varchar(50), departing_time TIME,
        duration varchar(20), reaching_time TIME, star_rating float(2,1), price DECIMAL(7, 2), seats_available INT)"""
cursor.execute(query)

query = """insert into All_bus_data(route_name, route_link, busname, bustype, departing_time, duration, reaching_time, star_rating, price, seats_available) 
        values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
for i in range(len(df)):
    data = data = (
        str(df.loc[i, 'route_name']),          
        str(df.loc[i, 'route_link']),          
        str(df.loc[i, 'busname']),          
        str(df.loc[i, 'bustype']),             
        str(df.loc[i, 'departing_time']),      
        str(df.loc[i, 'duration']),            
        str(df.loc[i, 'reaching_time']),       
        float(df.loc[i, 'star_rating']),       
        float(df.loc[i, 'price']),             
        int(df.loc[i, 'seats_available'])      
    )
    cursor.execute(query, data)
    con.commit()